In [3]:
#include("../DMSpecacim/DiscreteMaps.jl")
include("DiscreteMaps.jl")
using DiscreteMaps, Gadfly
DM = DiscreteMaps
M = 20
deps = 1e-6
Mp = [x=>DM.logistic(x(3.8,M*deps/20)) for x in [+,-]]
CO = [x=>DM.logisticcriticalorbit(Mp[x]) for x in [+,-]]
#COspds = 
mu = [x=>DM.spectralacim(Mp[x],200,uselogisticcofn=true) for x in [+,-]];

In [4]:
#Gadfly.plot([plotmeasure(mu[+];col=color("blue")),
#    plotmeasure(mu[-];col=color("green"))],Scale.x_continuous(minvalue=0.58,maxvalue=0.6))
N = 10^8
xh = [x=>DM.iterate(Mp[x],10^8) for x in [+,-]]

Dict{Function,Any} with 2 entries:
  - => [0.7069869242067932 0.7871941630317316 0.6365739808184095 0.879120450153…
  + => [0.780632672754508 0.6507323225794378 0.8636633416130461 0.4474462188229…

In [5]:
meshf = 10000
boxsize = 0.9/meshf
hgd = linspace(0.05+1e-6,0.95-1e-6,meshf+1)
pgd = hgd[1:end-1] + boxsize/2
#xhcts = [x => hist(xh[x],hgd)[2]/boxsize/N for x in [+,-]]
mp = [x=>DM.measureint(hgd,mu[x])[1] * 10^8 for x in [+,-]];

In [ ]:
# can we see any good peaks by eye?
col = [(+)=>"orange",(-)=>"red"]
# this plot is a stand-in for iterating (I'm too lazy)
#plot([layer(x=pgd,y=mp[x]+sqrt(abs(mp[x])).*randn(meshf),Geom.line,Theme(default_color=color(col[x])))[1] for x in [+,-]]...)
plot([layer(x=xh[x],Geom.histogram,Theme(default_color=color(col[x])))[1] for x in [+,-]])

In [7]:
nbyeye = find(0.80 .< DiscreteMaps.criticalorbit(DiscreteMaps.logistic(3.8)).pts .< 0.82)[1]
depsm = M/2 * deps
nstar = - log(depsm) / 2log(mean([DM.lyapunov(Mp[x],mu[x]) for x in [+,-]])) |> round |> int
n_one = 1
nv = [n_one,nstar,nbyeye]

3-element Array{Int64,1}:
  1
 13
 22

In [ ]:
addprocs(4-nprocs())

@everywhere include("DiscreteMaps.jl")

@everywhere module expas
    using DiscreteMaps
    alpha = 3.8
    copts = DiscreteMaps.logisticcriticalorbit(DiscreteMaps.logistic(alpha)).pts |> vec
    gauA(stdv,ns) = [DiscreteMaps.gaussian(copts[i],std) for std in stdv,i in ns] |> vec
    logisticgs(stdv,ns;largs...) = DiscreteMaps.IterationSchema(DiscreteMaps.logisticp(alpha),"Lgs",gauA(stdv,ns);largs...)
    end

@everywhere setupcode = quote
    nv = [1,13,22]
    using DiscreteMaps
    

    using expas
    function peturbsample(M,deps,kr)
        N = 10^8
        NH = 10^2
        DiscreteMaps.timedsample(expas.logisticgs(2. .^(-[1:0.5:30]),nv,
            samplefn=DiscreteMaps.evensamplefn,samplefnargs=(deps),N=N,NH=NH),
            NP=M,NCycles=1,startstring="results/lrb/rbugs-$(deps)-$(kr)-")
    end
end
@everywhere eval(setupcode)
peturbsample(M,deps,1.);

((),) in module DiscreteMaps at /Users/johnwormell/DiscreteMaps/DiscreteMaps.jl:28 overwritten in module DiscreteMaps at /Users/johnwormell/DiscreteMaps/DiscreteMaps.jl:28.
((),) in module DiscreteMaps at /Users/johnwormell/DiscreteMaps/DiscreteMaps.jl:28 overwritten in module DiscreteMaps at /Users/johnwormell/DiscreteMaps/DiscreteMaps.jl:28.


In [2]:
include("DiscreteMaps.jl")
using HDF5, JLD, DiscreteMaps
L = load("results/lrb/rbugs-1.0e-6-1.0--Lgs-2015-07-12T10-00-00--22-47-s.h5");
rss = DiscreteMaps.checklinearresponse(L)[1]
chim = reshape(rss,(59,3))
#methods(reshape)
plot(x=2.^(-[1:0.5:30]),y=max(0,chim[:,1]-(M-2)),Scale.y_log10(maxvalue=10^10),Scale.x_log10,Geom.line)

LoadError: plot not defined
while loading In[2], in expression starting on line 7

In [188]:
# does the lyapunov exponent thing work?
x1 = rand(1)
x2 = x1 + 1e-7
rp = 10
for i = 1:rp
    Mp[+].f!(x1,Mp[+].params)
    Mp[+].f!(x2,Mp[+].params)
end
(abs(x2 - x1)/1e-7).^(1/2rp)

1-element Array{Float64,1}:
 1.16348

In [63]:
# does the lyapunov-speeds-alpha thing work?
include("DiscreteMaps.jl")
using DiscreteMaps
DM = DiscreteMaps
lyk = Float64[]
alphak = Float64[]
alspdk = Float64[]
x = [1:100]
for k in 3.75:0.001:3.8
    Mpk = DM.logistic(k)
    COk = DM.logisticcriticalorbit(Mpk)
    y = COk.mag
    z = abs(DM.logisticcospeeds(COk,Mpk))
    push!(alphak, exp(-cov(x,log(y[:]))/var(x))[1])
    push!(lyk, DM.lyapunov(Mpk,DM.spectralacim(Mpk))[1])
    push!(alspdk, exp(-cov(x,log(z[:]))/var(x))[1])
end

In [14]:
[i for std in 2. .^[-1:0.5:30],i in nv] |> vec

189-element Array{Any,1}:
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  ⋮
 22
 22
 22
 22
 22
 22
 22
 22
 22
 22
 22
 22